Code to analyze gas temperature in a 2k bottle

In [14]:
import matplotlib as plt
import numpy as np
import scipy
from ctREFPROP.ctREFPROP import REFPROPFunctionLibrary
import os

# https://refprop-docs.readthedocs.io/en/latest/DLL/high_level.html#f/_/REFPROPdll

In [15]:
def fluid_P_D(fluid, p_pa, density):
    """
    Inputs:
        fluid: string of fluid to analyze 
        p_pa: pressure of the fluid, in Pascals
        density: density of the fluid, in kg/m^3
    
    Output: temperature of the gas
    """
    RP = REFPROPFunctionLibrary(os.environ['RPPREFIX'])
    RP.SETPATHdll(os.environ['RPPREFIX'])
    MOLAR_BASE_SI = RP.GETENUMdll(0,"MASS SI").iEnum
    r = RP.REFPROPdll(fluid,"PD","T",MOLAR_BASE_SI, 0,0,p_pa/1000000, density, [1.0])
    return r.Output[0]

def fluid_P_T(fluid, p_pa, temperature):
    """
    Inputs:
        fluid: string of fluid to analyze
        p_pa: pressure of the fluid, in Pascals
        temperature: temperature of the fluid, in Kelvin
    
    Output: density of the gas
    """
    RP = REFPROPFunctionLibrary(os.environ['RPPREFIX'])
    RP.SETPATHdll(os.environ['RPPREFIX'])
    MOLAR_BASE_SI = RP.GETENUMdll(0,"MASS SI").iEnum
    r = RP.REFPROPdll(fluid,"TP","D",MOLAR_BASE_SI, 0,0,temperature, p_pa/1000000, [1.0])
    return r.Output[0]

def convert_psi_to_pa(p_in_psi):
    # source: google
    return p_in_psi*6894.76

# If the RPPREFIX environment variable is not already set by your installer (e.g., on windows), 
# then uncomment this line and set the absolute path to the location of your install of 
# REFPROP
os.environ['RPPREFIX'] = r'C:\Program Files (x86)\REFPROP'

# Print the version of REFPROP in use and the NBP
out = fluid_P_D("NITROGEN", 101325, 1.183160685)
print(out)

288.62073043605136


In [16]:
volume = 0.040				# cubic meters
p_init = 2000				# psi
t_init = 280				# Kelvin
mdot_out = 0.105*1.05 		# kg/s
dt = 0.1					# seconds
fluid_to_use = "NITROGEN"	# name of fluid to use for simulation

In [17]:
# find initial density to find mass of gas in bottle
p_init_pa = convert_psi_to_pa(p_init)
density = fluid_P_T(fluid_to_use, p_init_pa, t_init)
mass = [density*volume]
gas_temp = t_init
i = 0
while gas_temp > 233.15:
	mass += [mass[i] - (mdot_out*dt)]
	

print(mass)


[6.616024234748961, 6.604999234748961, 6.604999234748961, 6.604999234748961, 6.604999234748961, 6.604999234748961]
165.40060586872403
